# Initial Exploration of Ferrybox Data

In this notebook, we

1. Load data from the ferrybox on the MS Color Fantasy ferry that sails from Oslo to Kiel.
2. Visualize the time-series of chlorophyill measurements.
3. Visualize the chlorophyill measurements along the track of the ferry.

The ferrybox data is available from the Thredds server hosted by NIVA.

See also https://thredds.niva.no/thredds/catalog/subcatalogs/ferryboxes.html.

## Preliminaries

In [1]:
%matplotlib inline

In [2]:
import xarray as xr
import geopandas as gpd
import pandas as pd
import movingpandas as mpd
import hvplot
from datetime import timedelta

## Load Data

In [3]:
url = "https://thredds.niva.no/thredds/dodsC/datasets/nrt/color_fantasy.nc"
ds = xr.open_dataset(url, engine='netcdf4')
ds

<xarray.Dataset> Size: 38MB
Dimensions:          (time: 489353)
Coordinates:
    latitude         (time) float64 4MB ...
    longitude        (time) float64 4MB ...
  * time             (time) datetime64[ns] 4MB 2023-01-01T13:16:00 ... 2024-0...
Data variables: (12/13)
    trajectory_name  |S64 64B ...
    temperature      (time) float64 4MB ...
    salinity         (time) float64 4MB ...
    oxygen_sat       (time) float64 4MB ...
    chlorophyll      (time) float64 4MB ...
    turbidity        (time) float64 4MB ...
    ...               ...
    temperature_qc   (time) int8 489kB ...
    salinity_qc      (time) int8 489kB ...
    oxygen_sat_qc    (time) int8 489kB ...
    chlorophyll_qc   (time) int8 489kB ...
    turbidity_qc     (time) int8 489kB ...
    fdom_qc          (time) int8 489kB ...
Attributes: (12/41)
    id:                                        no.niva:af11ba01-dfe3-4432-b9d...
    featureType:                               trajectory
    title:                                     FerryBox on MS Color Fantasy, ...
    summary:                                   Daily updating dataset of ferr...
    title_no:                                  FerryBox på MS Color Fantasy, ...
    summary_no:                                Daglig updatert dataset fra fe...
    ...                                        ...
    publisher_institution:                     Norwegian Institute for Water ...
    license:                                   http://spdx.org/licenses/CC-BY...
    iso_topic_category:                        oceans
    history:                                   Built with dscreator
    source:                                    FerryBox
    THREDDSMetadata_services_opendap_service:  https://thredds.niva.no/thredd...

## Select Time Slice

In [4]:
tlo = '2024-06-20T1200'
thi = '2024-06-23T1200'
ds = ds.sel(time=slice(tlo, thi))

## Transform to Geodataframe

In [5]:
# df = ds.to_dataframe().reset_index()  # timestamp as column, integer as index
df = ds.to_dataframe()  # timestamp as index

In [6]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs=4326)

In [7]:
gdf.head(4).T

time,2024-06-20 12:11:00,2024-06-20 12:12:00,2024-06-20 12:13:00,2024-06-20 12:14:00
trajectory_name,b'color_fantasy',b'color_fantasy',b'color_fantasy',b'color_fantasy'
latitude,59.876733,59.873063,59.86932,59.86561
longitude,10.646993,10.639557,10.63228,10.624892
temperature,18.12,18.16,18.3,18.19
salinity,13.654,13.7,13.886,13.95
oxygen_sat,115.54,116.1,116.32,115.99
chlorophyll,7.213888,8.209279,7.851225,6.576551
turbidity,1.612337,1.746771,1.631095,1.593579
fdom,26.196099,26.60672,26.241723,25.785477
temperature_qc,1,1,1,1


## Transform to Trajectory (using Moving Pandas)

In [8]:
traj = mpd.Trajectory(gdf, traj_id=1)
traj

Trajectory 1 (2024-06-20 12:11:00 to 2024-06-23 06:59:00) | Size: 3357 | Length: 2021029.4m
Bounds: (10.199983, 54.399123, 11.850723, 59.89945)
LINESTRING (10.646993 59.876733, 10.639557 59.873063, 10.63228 59.86932, 10.624892 59.86561, 10.6175

In [9]:
# When the ferry arrives, waits, and starts again, we start a new trajectory
traj_collection = mpd.ObservationGapSplitter(traj).split(gap=timedelta(hours=4))

In [10]:
# We can also export the trajectories as individual GeoDataFrames
# NB: We don't use this below
gdf_collection = traj_collection.to_traj_gdf()
gdf_collection.T

,0,1,2
traj_id,1_0,1_1,1_2
start_t,2024-06-20 12:11:00,2024-06-21 12:45:00,2024-06-22 12:20:00
end_t,2024-06-21 07:01:00,2024-06-22 07:41:00,2024-06-23 06:59:00
geometry,"LINESTRING (10.646993 59.876733, 10.639557 59....","LINESTRING (10.216915 54.417852, 10.220267 54....","LINESTRING (10.654208 59.879695, 10.647682 59...."
length,671008.712167,670178.725435,674015.715341
direction,182.718901,2.553673,182.766839


## Show Trajectory with Coloured Cholorophyill Measurements

You need to actually execute this notebook for the plot to show. If you just open it on Github/Gitlab, you won't see anything.

You can, however, open the file `./01_trajectories_with_data_sample.html`.

In [11]:
p = traj_collection.hvplot(tiles="CartoLight",
                           c="chlorophyll",
                           colormap='viridis',
                           line_width=4)

In [12]:
hvplot.save(p, '01_trajectories_with_data_sample.html')

In [13]:
p

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Path.I   :Path   [Longitude,Latitude]   (chlorophyll)
   .Points.I :Points   [Longitude,Latitude]   (chlorophyll,triangle_angle)